# Import Packages

In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy

### Get the time rounded down to the last 30 min interval

In [2]:
def rounded_to_the_last_30_minute():
    now = datetime.now()
    rounded = now - (now - datetime.min) % timedelta(minutes=30)
    return rounded

In [3]:
date = rounded_to_the_last_30_minute()
date

datetime.datetime(2023, 10, 23, 18, 0)

In [4]:
YYYYMMDD_HHMM = date.strftime('%Y%m%d_%H%M')
YYYYMMDD_HHMM

'20231023_1800'

### Import latest NDFD

In [5]:
File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"
File

'https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_20231023_1800.grib2'

In [6]:
ds = xr.open_dataset(File)
ds

/knight/mamba_aug23/envs/aug23_env/lib/python3.11/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'gribberish' loading failed:
/lib64/libc.so.6: version `GLIBC_2.28' not found (required by /nfs/home11/staff/ktyle/src/gribberish/python/gribberish/gribberishpy.cpython-311-x86_64-linux-gnu.so)
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:                                                                   (
                                                                               x: 2145,
                                                                               y: 1377,
                                                                               time: 58,
                                                                               time1: 13,
                                                                               time2: 6,
                                                                               ...
                                                                               height_above_ground: 1,
                                                                               height_above_ground1: 1,
                                                                               time1_bounds_1: 2,
                                                                               time2_bounds_1: 2,
                                                                               time5_bounds_1: 2,
                                                                               time6_bounds_1: 2)
Coordinates:
  * x                                                                         (x) float32 ...
  * y                                                                         (y) float32 ...
    reftime                                                                   datetime64[ns] ...
  * time                                                                      (time) datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time3                                                                     (time3) datetime64[ns] ...
  * time4                                                                     (time4) datetime64[ns] ...
  * time5                                                                     (time5) datetime64[ns] ...
  * time6                                                                     (time6) datetime64[ns] ...
  * height_above_ground                                                       (height_above_ground) float32 ...
  * height_above_ground1                                                      (height_above_ground1) float32 ...
Dimensions without coordinates: time1_bounds_1, time2_bounds_1, time5_bounds_1,
                                time6_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time1_bounds                                                              (time1, time1_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time5_bounds                                                              (time5, time5_bounds_1) datetime64[ns] ...
    time6_bounds                                                              (time6, time6_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time6, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                            (time, height_above_ground, y, x) float32 ...
    Wind_

In [7]:
ds = ds.metpy.parse_cf()
ds = ds.metpy.assign_latitude_longitude(force=False)
ds

<xarray.Dataset>
Dimensions:                                                                   (
                                                                               time1: 13,
                                                                               time1_bounds_1: 2,
                                                                               time2: 6,
                                                                               time2_bounds_1: 2,
                                                                               time5: 7,
                                                                               ...
                                                                               y: 1377,
                                                                               time: 58,
                                                                               height_above_ground1: 1,
                                                                               time4: 54,
                                                                               height_above_ground: 1,
                                                                               time3: 42)
Coordinates: (12/15)
    reftime                                                                   datetime64[ns] ...
  * time1                                                                     (time1) datetime64[ns] ...
  * time2                                                                     (time2) datetime64[ns] ...
  * time5                                                                     (time5) datetime64[ns] ...
  * time6                                                                     (time6) datetime64[ns] ...
  * x                                                                         (x) float32 ...
    ...                                                                        ...
    metpy_crs                                                                 object ...
  * time4                                                                     (time4) datetime64[ns] ...
  * height_above_ground                                                       (height_above_ground) float32 ...
  * time3                                                                     (time3) datetime64[ns] ...
    latitude                                                                  (y, x) float64 ...
    longitude                                                                 (y, x) float64 ...
Dimensions without coordinates: time1_bounds_1, time2_bounds_1, time5_bounds_1,
                                time6_bounds_1
Data variables: (12/20)
    LambertConformal_Projection                                               int32 ...
    time1_bounds                                                              (time1, time1_bounds_1) datetime64[ns] ...
    time2_bounds                                                              (time2, time2_bounds_1) datetime64[ns] ...
    time5_bounds                                                              (time5, time5_bounds_1) datetime64[ns] ...
    time6_bounds                                                              (time6, time6_bounds_1) datetime64[ns] ...
    Dewpoint_temperature_height_above_ground                                  (time, height_above_ground1, y, x) float32 ...
    ...                                                                        ...
    Total_snowfall_surface_6_Hour_Accumulation                                (time6, y, x) float32 ...
    Apparent_Temperature_height_above_ground                                  (time, height_above_ground1, y, x) float32 ...
    Weather_string_surface                                                    (time, y, x) float32 ...
    Wind_direction_from_which_blowing_height_above_ground                     (time, height_above_ground, y, x) float32 ...
    Wind_speed_height_above_ground                                            (time, 

In [8]:
latitude = ds.latitude
longitude = ds.longitude

In [9]:
ds = ds.sel(longitude=slice(-70,-60))
ds

KeyError: "no index found for coordinate 'longitude'"

In [ ]:
max_temp = ds.Maximum_temperature_height_above_ground_12_Hour_Maximum
max_temp

In [ ]:
max_coords = max_temp.coords
if "time" in max_coords:
    forecast_max = max_temp.isel(time=slice(None, 2)).compute()
    print('its time!')
elif "time1" in max_coords:
    forecast_max = max_temp.isel(time1=slice(None, 2)).compute()
    print('its time1!')
elif "time2" in max_coords:
    forecast_max = max_temp.isel(time2=slice(None, 2)).compute()
    print('its time2!')
elif "time3" in max_coords:
    forecast_max = max_temp.isel(time3=slice(None, 2)).compute()
    print('its time3!')
elif "time4" in max_coords:
    forecast_max = max_temp.isel(time4=slice(None, 2)).compute()
    print('its time4!')

In [ ]:
forecast_max = forecast_max.isel(height_above_ground1=0).compute()

In [ ]:
min_temp = ds.Minimum_temperature_height_above_ground_12_Hour_Minimum
min_temp

In [ ]:
min_coords = min_temp.coords
if "time" in min_coords:
    forecast_min = min_temp.isel(time=slice(None, 2)).compute()
    print('its time!')
elif "time1" in min_coords:
    forecast_min = min_temp.isel(time1=slice(None, 2)).compute()
    print('its time1!')
elif "time2" in min_coords:
    forecast_min = min_temp.isel(time2=slice(None, 2)).compute()
    print('its time2!')
elif "time3" in min_coords:
    forecast_min = min_temp.isel(time3=slice(None, 2)).compute()
    print('its time3!')
elif "time4" in min_coords:
    forecast_min = min_temp.isel(time4=slice(None, 2)).compute()
    print('its time4!')

In [ ]:
forecast_min = min_temp.isel(height_above_ground1=0).compute()